In [1]:
import pymongo
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tqdm import tqdm
import talib
import gc
from typing import Union
import argparse
from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt
import warnings
from IPython.display import display
warnings.filterwarnings("ignore")

In [2]:
class Chast:

    def __init__(self, client, db_name='Fields'):
        self.client = client
        self.db = client[db_name]
        self._config()
        self.data = dict()
        self.results = dict()
        self.initial_capital = 100000000
        
    def _config(self):
        # 設定繪圖、TQDM讀取條、小數點位數、顯示視窗長度
        pd.options.plotting.backend = "matplotlib"
        tqdm.pandas(desc="progress-bar")
        plt.rcParams['font.family'] = ['Microsoft JhengHei'] # 中文標籤
        plt.rcParams['axes.unicode_minus'] = False # 負號
        plt.rcParams['figure.figsize'] = (16,12)
        plt.rcParams.update({'font.size': 12})
        pd.set_option('display.max_rows', 200)
        pd.set_option('display.float_format', lambda x: '%.3f' % x)
        pd.options.display.float_format = '{:,.4f}'.format

    def _result(self):
        pf = self.results['PROFIT']
        com = self.results['COMMISION']
        self.results['RESULT'] = (pf - com).sum(axis=1).cumsum()

    def _maxdrawdown(self):
        se = self.results['RESULT']
        mdd = 0
        mdd_ = list()
        h = 0
        for i in se:
            if i > h:
                h = i
            mdd = i - h
            mdd_.append(mdd)
        self.results['MDD'] = pd.Series(mdd_, index=se.index, name='MDD')

    def _sharpe_ratio(self, roll: int = 120):
        """
        計算Sharpe ratio
        預設roll 120天=半年
        roll
        """
        pf = self.results['RESULT'] 
        self.results['SHARPE RATIO'] = (pf-pf.shift(1)).rolling(roll).apply(lambda x: x.mean() / x.std() if x.std() != 0 else 0, raw=True)
    
    def _holding(self):

        bs = self.results['BS']
        if '收盤價' in self.data.keys():
            holding = bs.fillna(0) * self.data['收盤價'].loc[bs.index, bs.columns] * 1000
        else:
            self.get_from_mongo('收盤價')
            holding = bs.fillna(0) * self.data['收盤價'].loc[bs.index, bs.columns] * 1000
        self.results['HOLDING'] = holding

    def _capital(self):
        self.results['CAPITAL NET'] = self.results['CAPITAL'][:-1]

    def set_result(self):
        self._result()
        self._maxdrawdown()
        self._holding()
        self._sharpe_ratio(120)
        self._capital()

    def get_chart(self):
        keys = self.results
        if 'RESULT' not in keys:
            self.set_result()
        keys = self.results
        fig ,ax = plt.subplots(8, 1, figsize=(16, 16), gridspec_kw={'height_ratios': [2, 1, 1, 1, 1, 1, 2, 1]})
        
        if 'RESULT' in keys:
            ax[0].plot(self.results['RESULT'][:-1])
            ax[0].set_title(f'累計損益')
            ax[0].grid(True)
            ax[0].yaxis.set_major_formatter('{x:,.0f}')
            ax[0].xaxis.set_minor_locator(mdates.YearLocator())
        if 'MDD' in keys:
            ax[1].plot(self.results['MDD'][:-1], color='orange')
            ax[1].grid(True)
            ax[1].set_title('MaxDrawdown')
            ax[1].yaxis.set_major_formatter('{x:,.0f}')
        if 'BS' in keys:
            ax[2].plot(self.results['BS'].fillna(0).astype(bool).sum(axis=1)[:-1], color='green')
            ax[2].set_title('累計股數')
        if 'HOLDING' in keys:
            ax[3].plot(self.results['HOLDING'].sum(axis=1)[:-1], color='lightgreen')
            ax[3].set_title('累計市值')
            ax[3].yaxis.set_major_formatter('{x:,.0f}')
        if 'SHARPE RATIO' in keys:
            ax[4].plot(self.results['SHARPE RATIO'][:-1], color='lightblue')
            ax[4].grid(True)
            ax[4].set_title('Rolling Sharpe Ratio')
            ax[4].yaxis.set_major_formatter('{x:.3f}')
        if 'CAPITAL' in keys:
            ax[5].plot(self.results['CAPITAL NET'][:-1], color='darkgreen')
            ax[5].grid(True)
            ax[5].set_title('Capital')
            ax[5].yaxis.set_major_formatter('{x:,.0f}')
        if 'HEDGE' in keys:
            ax[6].plot(self.results['RESULT'] + self.results['HEDGE'].cumsum())
            ax[6].grid(True)
            ax[6].set_title('避險後損益')
            ax[6].yaxis.set_major_formatter('{x:,.0f}')
            ax[7].plot(self.results['HEDGE'].cumsum())
            ax[7].grid(True)
            ax[7].set_title('避險單獨損益')
            ax[7].yaxis.set_major_formatter('{x:,.0f}')

        fig.tight_layout()
        plt.show()        

    def get_from_mongo(self, elements, nodate: bool=False,
     start: Union[datetime.datetime, None]=None, 
     end: Union[datetime.datetime, None]=None):
        """
        elements: Element, 表示要在Mongo中抓哪些資料, 可用list包起來好幾項
        start: 開始時間
        end: 結束時間
        """
        if type(elements) != list:
            if elements not in self.data:
                if nodate:
                    self.data[elements] = pd.DataFrame(self.db[elements].find({}, {'_id': 0}))
                else:
                    self.data[elements] = pd.DataFrame(self.db[elements].find({"日期": {'$gt': start, '$lt': end}}, {'_id': 0})).set_index('日期')
        else:
            for e in elements:
                if e not in self.data:
                    if nodate:
                        self.data[e] = pd.DataFrame(self.db[e].find({}, {'_id': 0}))
                    else:
                        self.data[e] = pd.DataFrame(self.db[e].find({"日期": {'$gt': start, '$lt': end}}, {'_id': 0})).set_index('日期')

    def set_data(self, name: str, df: pd.DataFrame):
        """
        name: 指定data名稱
        data: 放入data
        """
        self.data[name] = df
    
    def set_result(self, pf: pd.DataFrame, bs: pd.DataFrame, com: pd.DataFrame, ep: pd.DataFrame, ed: pd.DataFrame, ca: pd.Series, hedge: pd.Series=None, hbs: pd.Series=None):
        """
        pf: 損益明細
        bs: 部位明細
        com: 費用明細
        ep: 進場價格明細
        ed: 進場時間明細
        ca: 剩餘資金變化
        hedge: 避險損益
        """
        self.results['PROFIT'] = pf
        self.results['BS'] = bs
        self.results['COMMISION'] = com
        self.results['ENTRY PRICE'] = ep
        self.results['ENTRY DATE'] = ed
        self.results['CAPITAL'] = ca
        if hedge != None:
            self.results['HEDGE'] = hedge
        if hbs != None:
            self.results['HBS'] = hbs
        
    def creat_to_daily(self, df: pd.DataFrame):
        n = pd.DataFrame(columns=self.data['還原收盤價'].columns, index=self.data['還原收盤價'].index.union(df.index))
        for i in df.index:
            for c in df.columns:
                n.at[i, str(c)] = df.at[i, c]
        return n.fillna(method='ffill')

In [3]:

client = pymongo.MongoClient()
chast = Chast(client, 'Fields')
index = Chast(client, 'Index')
db1 = client['Index']

### <font color=osheet.range> 策略步驟 </font>
* 1. 下載需要的資料
* 2. 設定要用到的資料，時間點要對齊收盤
* 3.

##### 1. 下載需要的資料

In [4]:
getting_list = [
    '本益比(近四季)', '股價淨值比', '還原收盤價', '成交金額(千)', '殖利率',
    'EPS_Q', '營業收入淨額_Q', '收盤價', '還原開盤價', '還原最高價', '還原最低價', '收盤價',
]

start = datetime.datetime(2004, 1, 1)

end = datetime.datetime.today()
# 原始資料
chast.get_from_mongo(getting_list, start=start, end=end)
chast.set_data('收盤價', chast.data['收盤價'].fillna(method='ffill'))
chast.set_data('還原收盤價', chast.data['還原收盤價'].fillna(method='ffill'))
chast.set_data('MA60', chast.data['還原收盤價'].rolling(60).mean())
chast.set_data('MA120', chast.data['還原收盤價'].rolling(120).mean())
# chast.set_data('大盤收盤價', pd.DataFrame(db1['TWA00'].find({"日期": {'$gt': start, '$lt': end}}, {'_id': 0, '日期': 1, '還原收盤價': 1})).set_index('日期').dropna(how='all'))

# 調整後資料
# log收盤價 = chast.data['還原收盤價'].copy()
# for c in log收盤價.columns:
#     if log收盤價.loc[:, c].sum() != 0:
#         log收盤價.loc[:, c] = np.log(log收盤價.loc[:, c])
# chast.set_data('log收盤價', log收盤價)
chast.set_data('EPS_daily', chast.creat_to_daily(chast.data['EPS_Q'].rolling(4).sum().pct_change().replace([np.inf, -np.inf], np.nan)))
chast.set_data('營業收入淨額_daily', chast.creat_to_daily(chast.data['營業收入淨額_Q'].rolling(4).sum().pct_change().replace([np.inf, -np.inf], np.nan)))
std_level = chast.data['還原收盤價'].pct_change().rolling(20).std().div(chast.data['還原收盤價'].pct_change().std(axis=1), axis=0).replace([np.inf, -np.inf], 1)
for i, v in std_level.iterrows():
    # 去除極端值
    v99 = v.quantile(.95)
    v01 = v.quantile(.05)
    mask_99 = v[v > v99].index
    mask_01 = v[v < v01].index
    std_level.loc[i, mask_99] = v99
    std_level.loc[i, mask_01] = v01

# 調整讓所有資料的columns一樣
idx_, col_ = chast.data['還原收盤價'].index, chast.data['還原收盤價'].columns

for k in chast.data.keys():
    if k in ['EPS_Q', '營業收入淨額_Q', '預估EPS_Q', '營業收入淨額_Y']:
        continue
    if k in ['大盤收盤價']:
        idx_ = idx_.intersection(chast.data[k].index)
    else:
        idx_ = idx_.intersection(chast.data[k].index)
        col_ = col_.intersection(chast.data[k].columns)
for k in chast.data.keys():
    if k in ['EPS_Q', '營業收入淨額_Q', '預估EPS_Q', '營業收入淨額_Y']:
        continue
    if k in ['大盤收盤價']:
        chast.data[k] = chast.data[k].loc[idx_]
    else:
        chast.data[k] = chast.data[k].loc[idx_, col_]


# %%
# 極端值處理
for n in ['EPS_daily', '本益比(近四季)', '營業收入淨額_daily', '股價淨值比', '殖利率']:
    for idx, row in chast.data[n].iterrows():
        mask = row[row > row.quantile(.99)].index
        chast.data[n].loc[idx, mask] = row.quantile(.99)
        mask = row[row < row.quantile(.01)].index
        chast.data[n].loc[idx, mask] = row.quantile(.01)

# %%
#{k: v.shape for k, v in chast.data.items()}
{k: v.index[0] for k, v in chast.data.items()}

# %%
# long = (chast.data['本益比(近四季)'] < 15)&\
#     (chast.data['股價淨值比'] < 1.5)&\
#     (chast.data['成交金額(千)'].ge(chast.data['成交金額(千)'].median(axis=1), axis=0))&\
#     (chast.data['還原收盤價'] > chast.data['MA60'])&\
#     (chast.data['還原收盤價'] > chast.data['MA120'])&\
#     (chast.data['EPS_daily'] > 0)&\
#     (chast.data['營業收入淨額_daily'] > 0)&\
#     (chast.data['殖利率'] > 4)
# long = long.shift(1)

# # %%
# short = (chast.data['本益比(近四季)'] > 15)&\
#     (chast.data['股價淨值比'] > 1.5)&\
#     (chast.data['成交金額(千)'].ge(chast.data['成交金額(千)'].median(axis=1), axis=0))&\
#     (chast.data['還原收盤價'] < chast.data['MA60'])&\
#     (chast.data['還原收盤價'] < chast.data['MA120'])&\
#     (chast.data['EPS_daily'] < 0)&\
#     (chast.data['營業收入淨額_daily'] < 0)&\
#     (chast.data['殖利率'] < 2)
# short = short.shift(1)

{'本益比(近四季)': Timestamp('2004-01-02 00:00:00'),
 '股價淨值比': Timestamp('2004-01-02 00:00:00'),
 '還原收盤價': Timestamp('2004-01-02 00:00:00'),
 '成交金額(千)': Timestamp('2004-01-02 00:00:00'),
 '殖利率': Timestamp('2004-01-02 00:00:00'),
 'EPS_Q': Timestamp('2004-03-31 00:00:00'),
 '營業收入淨額_Q': Timestamp('2004-03-31 00:00:00'),
 '收盤價': Timestamp('2004-01-02 00:00:00'),
 '還原開盤價': Timestamp('2004-01-02 00:00:00'),
 '還原最高價': Timestamp('2004-01-02 00:00:00'),
 '還原最低價': Timestamp('2004-01-02 00:00:00'),
 'MA60': Timestamp('2004-01-02 00:00:00'),
 'MA120': Timestamp('2004-01-02 00:00:00'),
 'EPS_daily': Timestamp('2004-01-02 00:00:00'),
 '營業收入淨額_daily': Timestamp('2004-01-02 00:00:00')}

In [5]:
getting_list = [
    '還原開盤價', '還原最高價', '還原最低價', '還原收盤價', '收盤價', '成交金額(千)', '總市值(億)', '漲幅(%)'
]
#start = datetime.datetime(2018, 1, 1)
# end = datetime.datetime(2022, 10, 11)
#end = datetime.datetime.today()
chast.get_from_mongo(getting_list, start=start, end=end)

index_list = [
    'TWA00', 'VIX'
]
index.get_from_mongo(index_list, start=start, end=end)
index.get_from_mongo('產業名稱', nodate=True)


##### 2. 設定要用到的資料

In [6]:
data_list = [
    chast.data['還原開盤價'],
    chast.data['還原最高價'],
    chast.data['還原最低價'],
    chast.data['還原收盤價'],
    chast.data['成交金額(千)'],
    chast.data['總市值(億)'],
    chast.data['本益比(近四季)'],
    chast.data['股價淨值比'],
    chast.data['殖利率'],
    chast.data['EPS_daily'],
    chast.data['營業收入淨額_daily'],
    chast.data['MA60'],
    chast.data['MA120'],
]

for d in data_list:
    print(d.shape)

(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)
(4661, 2211)


In [7]:
value = {}
for c in chast.data['還原收盤價'].columns:
    d_list = [d.loc[:, c] for d in data_list]
    value[c] = pd.concat(d_list, axis=1)
    value[c].columns = ['open', 'high', 'low', 'close', 'volume', 'openinterset', 'pe', 'pb', 'yield', 'eps', 'earning', 'ma60', 'ma120']

In [8]:
print(len(value))
remove_list = list()
for k, v in value.items():
    print(k, v.isna().sum().sum() / (v.shape[0] * v.shape[1]))
    if v.isna().sum().sum() / (v.shape[0] * v.shape[1]) == 1:
        remove_list.append(k)
for k in remove_list:
    del value[k]
print(len(value))

2211
2706 0.01303780964797914
8016 0.017972373046391498
8719 1.0
4904 0.014754179525687786
2882 0.01303780964797914
6441 0.7387652039014407
1407 0.7632729853283382
2316 0.03168682851154424
6182 0.04158896242140181
1785 0.09468090373475484
4534 0.04592939778522272
3576 0.3132209991253115
3541 0.3162081428547852
6179 0.04843793837571997
5817 1.0
1229 0.01303780964797914
3698 0.5806446289175318
1453 0.056607198851352465
5608 0.035433135840773684
8279 0.6767943491822488
8240 0.10506164078358886
1781 0.11605300942353076
6122 0.03365075173699932
1227 0.01371445546515274
2469 0.6557193075107686
1904 0.01303780964797914
3686 0.39445150429917647
6405 0.5367121614707969
8404 0.40171306916640537
2807 0.6751274899740894
4162 0.47292591553479774
2905 0.024722327661611077
4116 0.636476160612612
6672 0.8104731569653261
6217 0.030102487085967026
3226 0.05728384466852607
9960 0.07167494595085241
4303 0.05401614047827307
9801 0.7038436783126764
8114 0.09892231775947717
3038 0.03855230802237882
9902 0.05

##### 將各產業分開來，並且計算每天的漲跌

index.data['產業名稱']['股票代號'] = index.data['產業名稱']['股票代號'].astype(str)
value = {}

for i in index.data['產業名稱']['產業名稱'].unique():
    ticker = index.data['產業名稱'][index.data['產業名稱']['產業名稱'] == i]['股票代號']
    se1 = (chast.data['總市值(億)'].loc[:, ticker] * chast.data['漲幅(%)'] * 0.01).sum(axis=1) / chast.data['總市值(億)'].loc[:, ticker].sum(axis=1)
    se1 = se1.cumsum() * 10000
    se1.fillna(0, inplace=True)
    se1.name = 'close'
    se2 = chast.data['成交金額(千)'].loc[:, ticker].sum(axis=1)
    se2.name = 'volume'
    value[i] = pd.concat([se1, se2], axis=1)



##### 現在我擁有各產業的市值累計值，直接以這個資料來測試簡單突破

In [9]:


class difInd(bt.Indicator):
    lines = ('dif',)

    def next(self):
        self.lines.dif[0] = self.datas[0].close[0] - self.datas[1].close[0]

class Strategy(bt.Strategy):

    params = (
        ('reserve', 0.05),
        ('selcperc', 0.05),
        ('stdperiod', 24),
        ('maperiod', 24),
        ('rolling_window', 130),
        ('tp_ratio', 20),
        ('sl_ratio', 6),
        ('longout', 80),
        ('shortout', 120),
        ('printout', False),

    )
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        print('prepared long...')
        self.long = {d: bt.And(d.pe<15, d.pb<1.5, d.close>d.ma60, d.close>d.ma120, d.eps>0, d.earning>0, d.y>4) for d in self.datas}
        print('long ok\nprepared short...')
        self.short = {d: bt.And(d.pe>15, d.pb>1.5, d.close<d.ma60, d.close<d.ma120, d.eps<0, d.earning<0, d.y<2) for d in self.datas}
        print('short ok')
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.sellprice = None
        self.sellcomm = None

    def start(self):
        if self.p.printout:
            pass


    def notify_order(self, order):
        type = "Buy" if order.isbuy() else "Sell"

        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                if self.p.printout:
                    self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                        (order.executed.price,
                        order.executed.value,
                        order.executed.comm))
                    pass
                    
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            elif order.issell():
                if self.p.printout:
                    self.log(
                        'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                        (order.executed.price,
                        order.executed.value,
                        order.executed.comm))
                    pass
                    
                self.sellprice = order.executed.price
                self.sellcomm = order.executed.comm

            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def nodify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                (trade.pnl, trade.pnlcomm))


    def next(self):
        # self.log('Close,%s %.2f %d' % (self.data._name, self.data.close[0], self.long[0]))

        if self.order:
            return
        long = {d for d, v in self.long.items() if v}
        short = {d for d, v in self.short.items() if v}

        posdata = [d for d, pos in self.getpositions().items() if pos]
        # self.log('Posdata: {}'.format([d._name for d in posdata]))
        # self.log('long: {}'.format([d._name for d in long]))
        # self.log('short: {}'.format([d._name for d in short]))    
        if len(self) % 20 == 0:
            for d in (d for d in long if d not in posdata):
                if (np.isnan(d.close[0]) == False) & (np.isnan(d.close[1]) == False):
                    if self.p.printout:
                        self.log('BUY CREATE, %s %.2f Have CASH: %.2f' % (d._name, d.close[0], self.broker.get_cash()))
                    self.order = self.buy(d, exectype=bt.Order.Close, size=int(self.broker.get_cash()*0.005/d.close[0]))

            for d in (d for d in posdata if (d not in long) & (d not in short)):
                if (np.isnan(d.close[0]) == False) & (np.isnan(d.close[1]) == False):
                    self.log('Exit: {}'.format(d._name))
                    self.order = self.order_target_percent(d, exectype=bt.Order.Close, target=0.0)

            for d in (d for d in short if d not in posdata) :
                if (np.isnan(d.close[0]) == False) * (np.isnan(d.close[1]) == False):
                    if self.p.printout:
                        self.log('SELL CREATE, %s %.2f' % (d._name, d.close[0]))
                    self.order = self.sell(d, exectype=bt.Order.Close, size=int(self.broker.get_cash()*0.0125/d.close[0]))






class difInd(bt.Indicator):
    lines = ('dif',)

    def next(self):
        self.lines.dif[0] = self.datas[0].close[0] - self.datas[1].close[0]

class Strategy(bt.Strategy):

    params = (
        ('reserve', 0.05),
        ('selcperc', 0.05),
        ('stdperiod', 24),
        ('maperiod', 24),
        ('rolling_window', 130),
        ('tp_ratio', 20),
        ('sl_ratio', 6),
        ('longout', 80),
        ('shortout', 120),
        ('printout', False),

    )
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
    
    def __init__(self):
        std = [bt.ind.StdDev(d, period=self.p.stdperiod) for d in self.datas]
        ma = [bt.ind.MovingAverageSimple(d, period=self.p.maperiod) for d in self.datas]
        dirty = [d > 0 for d in self.datas]
        self.crossover = {d: bt.And((d.close > (m+s)), (dir), (s>0)) for d, m, s, dir in zip(self.datas, ma, std, dirty)}
        self.crossunder = {d: bt.Or((d.close < (m-s)), (dir), (s==0)) for d, m, s, dir in zip(self.datas, ma, std, dirty)}
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.selnum = int(len(self.datas) * self.p.selcperc)
        self.perctarget = (1.0 - self.p.reserve) % self.selnum

    def start(self):
        if self.p.printout:
            pass


    def notify_order(self, order):
        type = "Buy" if order.isbuy() else "Sell"

        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                if self.p.printout:
                    # self.log(
                    #     'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    #     (order.executed.price,
                    #     order.executed.value,
                    #     order.executed.comm))
                    pass
                    
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            elif order.issell():
                if self.p.printout:
                    # self.log(
                    #     'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' % 
                    #     (order.executed.price,
                    #     order.executed.value,
                    #     order.executed.comm))
                    pass
            self.bar_executed = len(self)
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def nodify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                (trade.pnl, trade.pnlcomm))


    def next(self):
        # self.log('Close,%s %.2f' % (self.data._name, self.dataclose[0]))

        if self.order:
            return
        crossover = {d for d, v in self.crossover.items() if v}
        crossunder = {d for d, v in self.crossunder.items() if v}

        posdata = [d for d, pos in self.getpositions().items() if pos]
        self.log('Posdata: {}'.format([d._name for d in posdata]))
        self.log('Crossover: {}'.format([d._name for d in crossover]))        
        for d in (d for d in crossover if d not in posdata):
            if self.p.printout:
                pass
                #self.log('BUY CREATE, %s %.2f' % (d._name, d.close[0]))
            self.order = self.order_target_percent(d, target=self.perctarget)
            self.log('Buy Value {}'.format(self.order.executed.value))
            #self.order = self.buy(d, size=int(self.broker.get_cash()/d.close/50))

        for d in (d for d in posdata if d in crossunder):
            if self.p.printout:
                pass
                #self.log('SELL CREATE, %s %.2f' % (d._name, d.close[0]))
            self.order = self.order_target_percent(d, target=0.0)
            self.log('Sell Value {}'.format(self.order.executed.value))
            #self.order = self.sell(d)


In [10]:
def parse_args(args=None):

    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description='Sample for pivot point and cross plotting')

    parser.add_argument('--maperiod', required=False, action='store',
                        type = float, default= 15, choices=range(1,300),
                        help=('Set maperiod'))

    parser.add_argument('--rolling_window', required=False, action='store',
                        type = float, default= 130, choices=range(1,300),
                        help=('Set rolling_window'))

    parser.add_argument('--tp_ratio', required=False, action='store',
                        type = float, default= 20, choices=range(1,300),
                        help=('Set tp_ratio'))

    parser.add_argument('--sl_ratio', required=False, action='store',
                        type = float, default= 6, choices=range(1,300),
                        help=('Set sl_ratio'))

    parser.add_argument('--longout', required=False, action='store',
                        type = float, default= 80, choices=range(1,300),
                        help=('Set longout'))

    parser.add_argument('--shortout', required=False, action='store',
                        type = float, default= 120, choices=range(1,300),
                        help=('Set shortout'))

    parser.add_argument('--printout', required=False, action='store_true',
                        default= False,
                        help=('Print data lines'))

    parser.add_argument('--cash', required=False, action='store',
                        type=float, default=100000000.0,
                        help=('Cash to start with'))

    parser.add_argument('--comm', required=False, action='store',
                        type=float, default=0.002,
                        help=('Set Commission Rate'))

    parser.add_argument('--plot', required=False, action='store_true',
                        default= False, 
                        help=('Plot the result'))

    parser.add_argument('--plot-style', required=False, action='store',
                        default='bar', choices=['bar', 'candle', 'line'],
                        help=('Plot style'))

    parser.add_argument('--use-pyfolio', required=False, action='store_true',
                        default= False,
                        help=('Do pyfolio things'))

    if args == None:
        argss ,unknown = parser.parse_known_args()
    else:
        argss = parser.parse_args(args)
    return argss

In [11]:
def runstart(args):
    args = parse_args(args)

    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(args.cash)

    class feed_data(bt.feeds.PandasData):
        lines = ('pe', 'pb', 'y', 'eps', 'earning', 'ma60', 'ma120', )

        params = (
            ('fromdate', datetime.datetime(2004, 1, 1)),
            ('todate', datetime.datetime.today()),
            ('timeframe', bt.TimeFrame.Minutes),
            ('compression', 1),
            ('dtformat', ('%Y-%m-%d %H:%M:%S')),
            ('tmformat', ('%H:%M:%S')),
            ('open', 0),
            ('high', 1),
            ('low', 2),
            ('close', 3),
            ('volume', 4),
            ('openinterest', 5),
            ('pe', 6),
            ('pb', 7),
            ('y', 8),
            ('eps', 9),
            ('earning', 10),
            ('ma60', 11),
            ('ma120', 12),
        )

    for k, v in value.items():
        data = feed_data(dataname=v, name=k)
        # Add the Data Feed to Cerebro
        cerebro.adddata(data)

    cerebro.broker.setcommission(commission=args.comm)
    cerebro.addstrategy(Strategy, printout=args.printout)

    if args.use_pyfolio:
        cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

    results = cerebro.run()

    if  args.use_pyfolio:
        strat = results[0]
        pyfoliozer = strat.analyzers.getbyname('pyfolio')

        returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
        if args.printout:
            print('-- RETURNS')
            print(returns)
            print('-- POSITIONS')
            print(positions)
            print('-- TRANSACTIONS')
            print(transactions)
            print('-- GROSS LEVERAGE')
            print(gross_lev)


        import pyfolio as pf
        pf.create_full_tear_sheet(
            returns,
            positions=positions,
            transactions=transactions,
            live_start_date='2017-01-01',
            )
    if  args.plot:
        plt.rcParams['figure.figsize'] = 30, 20  # that's default image size for this interactive session
        cerebro.plot(style=args.plot_style)[0]

    return args


In [12]:
args = runstart(['--use-pyfolio', '--printout'])

prepared long...
long ok
prepared short...
short ok
2005-04-20, BUY CREATE, 2103 4.08 Have CASH: 100000000.00
2005-04-20, BUY CREATE, 5604 9.04 Have CASH: 100000000.00
2005-04-20, BUY CREATE, 1704 6.76 Have CASH: 100000000.00
2005-04-20, BUY CREATE, 5015 4.35 Have CASH: 100000000.00
2005-04-20, BUY CREATE, 5312 6.41 Have CASH: 100000000.00
2005-04-20, BUY CREATE, 5603 5.45 Have CASH: 100000000.00
2005-04-20, BUY CREATE, 2206 7.54 Have CASH: 100000000.00
2005-04-21, BUY EXECUTED, Price: 4.24, Cost: 519607.76, Comm 1039.22
2005-04-21, BUY EXECUTED, Price: 8.98, Cost: 496674.82, Comm 993.35
2005-04-21, BUY EXECUTED, Price: 6.80, Cost: 502955.20, Comm 1005.91
2005-04-21, BUY EXECUTED, Price: 4.33, Cost: 497698.86, Comm 995.40
2005-04-21, BUY EXECUTED, Price: 6.34, Cost: 494539.02, Comm 989.08
2005-04-21, BUY EXECUTED, Price: 5.28, Cost: 484403.04, Comm 968.81
2005-04-21, BUY EXECUTED, Price: 7.52, Cost: 498666.24, Comm 997.33
2005-05-19, BUY CREATE, 3018 96.94 Have CASH: 96498465.97
2005-0